> Note: The visualization of this notebook depends on usage of SAPIEN's `Viewer`, which is not available on Colab. **You need to run this notebook locally**. Besides this notebook, you can also find the latest SAPIEN tutorial at [SAPIEN's documentation](https://sapien.ucsd.edu/docs/latest/index.html).

# Motion Planning in SAPIEN

In this tutorial series, you will learn how to solve motion planning in SAPIEN.

# Motion Planning Tutorial 1: Plan a Path

In this tutorial, you will learn the following:

- Plan paths for the agent

> Note: This tutorial only talks about the basic usages, and the robot only avoids self-collisions (i.e., collisions between the robot links) in this demo. Please refer to *Motion Planning Tutorial 2: Collision Avoidance* to include the environment model and other advanced usages.

<a class="RRT" href="../assets/RRT.gif"><img alt="../assets/RRT.gif" src="../assets/RRT.gif" style="width: 33%;" /></a>
*Plan with RRTConnect*
<a class="screw" href="../assets/screw.gif"><img alt="../assets/screw.gif" src="../assets/screw.gif" style="width: 33%;" /></a>
*Plan with screw motion*

As shown in the demo, the robot needs to move the three boxes a bit forward.

## Preparation

We recommend using [mplib](https://pypi.org/project/mplib/) for motion planning with SAPIEN. `mplib` is a lightweight python package that includes common functionalities for motion planning. You can use `mplib` to plan a collision-free trajectory for a robot, calculate inverse kinematics, and take point cloud observation as an environment model.

In [ ]:
%pip install sapien
%pip install mplib

import sapien.core as sapien
import mplib
import numpy as np
from sapien.utils import Viewer

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    raise RuntimeError("The visualization of this notebook cannot be run on Colab! You need to run it locally with a GPU.")

Let's set up the scene we will be using in this tutorial and load the robot:

In [ ]:
# Initialize engine and renderer
engine = sapien.Engine()
engine.set_log_level('critical')
renderer = sapien.SapienRenderer()
engine.set_renderer(renderer)

# Build scene
scene_config = sapien.SceneConfig()
scene = engine.create_scene(scene_config)
scene.set_timestep(1 / 240.0)
scene.add_ground(-0.8)
physical_material = scene.create_physical_material(1, 1, 0.0)
scene.default_physical_material = physical_material

builder = scene.create_actor_builder()
builder.add_box_collision(half_size=[0.4, 0.4, 0.025])
builder.add_box_visual(half_size=[0.4, 0.4, 0.025])
table = builder.build_kinematic(name='table')
table.set_pose(sapien.Pose([0.56, 0, - 0.025]))

builder = scene.create_actor_builder()
builder.add_box_collision(half_size=[0.02, 0.02, 0.06])
builder.add_box_visual(half_size=[0.02, 0.02, 0.06], color=[1, 0, 0])
red_cube = builder.build(name='red_cube')
red_cube.set_pose(sapien.Pose([0.4, 0.3, 0.06]))

builder = scene.create_actor_builder()
builder.add_box_collision(half_size=[0.02, 0.02, 0.04])
builder.add_box_visual(half_size=[0.02, 0.02, 0.04], color=[0, 1, 0])
green_cube = builder.build(name='green_cube')
green_cube.set_pose(sapien.Pose([0.2, -0.3, 0.04]))

builder = scene.create_actor_builder()
builder.add_box_collision(half_size=[0.02, 0.02, 0.07])
builder.add_box_visual(half_size=[0.02, 0.02, 0.07], color=[0, 0, 1])
blue_cube = builder.build(name='blue_cube')
blue_cube.set_pose(sapien.Pose([0.6, 0.1, 0.07]))

# Set light for visualization
scene.set_ambient_light([0.5, 0.5, 0.5])
scene.add_directional_light([0, 1, -1], [0.5, 0.5, 0.5], shadow=True)
scene.add_point_light([1, 2, 2], [1, 1, 1], shadow=True)
scene.add_point_light([1, -2, 2], [1, 1, 1], shadow=True)
scene.add_point_light([-1, 0, 1], [1, 1, 1], shadow=True)

# Load robot
loader = scene.create_urdf_loader()
robot = loader.load("../assets/panda/panda.urdf")
robot.set_root_pose(sapien.Pose([0, 0, 0], [1, 0, 0, 0]))

# Set initial joint positions
init_qpos = [0, 0.19634954084936207, 0.0, -2.617993877991494, 0.0, 2.941592653589793, 0.7853981633974483, 0, 0]
robot.set_qpos(init_qpos)

## Planner Configuration

To use `mplib`, we need to set up a planner for the robot:

In [ ]:
link_names = [link.get_name() for link in robot.get_links()]
joint_names = [joint.get_name() for joint in robot.get_active_joints()]
planner = mplib.Planner(
    urdf="../assets/panda/panda.urdf",
    srdf="../assets/panda/panda.srdf",
    user_link_names=link_names,
    user_joint_names=joint_names,
    move_group="panda_hand",
    joint_vel_limits=np.ones(7),
    joint_acc_limits=np.ones(7)
)

- The URDF file describes the robot, while the [SRDF](http://docs.ros.org/en/kinetic/api/moveit_tutorials/html/doc/urdf_srdf/urdf_srdf_tutorial.html) file complements the URDF and specifies additional information for motion planning. For example, `mplib` loads the `disable_collisions` pairs in SRDF to ignore collisions between certain pairs of links. SRDF files are generated by `MoveIt Setup Assistant`.
- To align the link order and joint order between SAPIEN and `mplib`, we need to provide `user_link_names` and `user_joint_names`. After that, `mplib` can tell which joints the `qpos` vector (from SAPIEN) is referring to. Please note that we only care about the active joints (i.e., revolute and prismatic joints) and ignore the fixed joints.
- `move_group` specifies the target link for which we may specify target poses to reach. The end-effector of an agent is typically specified as the `move_group`. After specifying the `move_group`, `mplib` only focuses on those active joints along the path from the root link to the `move_group`, since other joints doesn’t affect the pose of the `move_group`. For example, for our `panda` robot arm (7 DoF), the end-effector is `panda_hand`. Only the first seven active joints affect the pose of `panda_hand`, while the last two finger joints don’t.
- For safety, the robot cannot move arbitrarily fast. `joint_vel_limits` and `joint_acc_limits` specify the maximum joint velocity and maximum joint acceleration constraints for the active joints along the path from the root link to the `move_group`. `mplib` takes the constraints into account when solving the time-optimal path parameterization.

After setting up the planner, we can use it to solve many motion planning tasks.

## Plan with sampling-based algorithms

`mplib` supports state-of-the-art sampling-based motion planning algorithms by leveraging [OMPL](https://github.com/ompl/ompl). You can call `planner.plan()` to plan a path for moving the `move_group` link to a target pose:

In [ ]:
def move_to_pose_with_RRTConnect(pose, viewer): # parameter viewer is for later visualization
    result = planner.plan(pose, robot.get_qpos(), time_step=1/250)
    if result['status'] != "Success":
        print(result['status'])
        return -1
    follow_path(result, viewer) # Function that control the robot to follow the planned path, We will define this later
    return 0

Specifically, `planner.plan()` takes two required arguments as input. The first one is the target pose of the `move_group` link. It’s a 7-dim list, where the first three elements describe the position part, and the remaining four elements describe the quaternion (wxyz) for the rotation part. **Note that the pose is relative to the frame of the robot’s root link**. The second argument is the current joint positions of all the active joints (e.g., given by SAPIEN). The `planner.plan()` function first solves the inverse kinematics to get the joint positions for the target pose. It then calls the RRTConnect algorithm to find a path in the joint space. Finally, it parameterizes the path to generate time, velocity, and acceleration information.

`planner.plan()` returns a dict which includes:

- `status`: a string indicates the status:
    - `Success`: planned a path successfully.
    - `IK Failed`: failed to solve the inverse kinematics. This may happen when the target pose is not reachable.
    - `RRT Failed`: failed to find a valid path in the joint space. This may happen when there is no valid path or the task is too complicated.
- `position`: a NumPy array of shape $n \times m$ describes the joint positions of the waypoints. $n$ is the number of waypoints in the path, and each row describes a waypoint. $m$ is the number of active joints that affect the pose of the `move_group` link. For example, for our panda robot arm, each row includes the positions for the first seven joints.
- `duration`: a scalar indicates the duration of the output path. `mplib` returns the optimal duration considering the velocity and acceleration constraints.
- `time`: a NumPy array of shape $n$ describes the time step of each waypoint. The first element is equal to 0, and the last one is equal to the `duration`. Argument `time_step` determines the interval of the elements.
- `velocity`: a NumPy array of shape $n \times m$ describing the joint velocities of the waypoints.
- `acceleration`: a NumPy array of shape $n \times m$ describing the joint accelerations of the waypoints.

`planner.plan()` also takes other optional arguments with default values:

- `time_step = 0.1`: `time_step` specify the time interval between the waypoints. The larger the value, the sparser the output waypoints. In this demo, we align the `time_step` with SAPIEN’s time step.
- `rrt_range = 0.1`: the incremental distance in the RRTConnect algorithm. The larger the value, the sparser the sampled waypoints (before time parameterization).
- `planning_time=1`: time limit for RRTConnect algorithm, in seconds.
- `fix_joint_limits=True`: whether to clip the current joint positions if they are out of the joint limits.
- `verbose=False`: whether to display some internal outputs.
- `use_point_cloud=False` and `use_attach=False`: related to collision avoidance, will be discussed in *Motion Planning Tutorial 2: Collision Avoidance*.

## Follow a path

`plan()` outputs a time-parameterized path, and we need to drive the robot to follow the path. If you have already finished *Robotics Tutorial 2: Drive Robot with PID Controller* (which is also recommended before starting this tutorial), you might have been familiar with the process of setting up a controller. Depending on your controller, you may only use the returned position information, or use the velocity and acceleration information as well.

In this demo, we use the PhysX internal PD controller. We first need to set the drive properties of the active joints at the very beginning:

In [ ]:
active_joints = robot.get_active_joints()
for joint in active_joints:
    joint.set_drive_property(stiffness=1000, damping=200)

To follow a path, at each time step, we set the target position and target velocity according to the returned path. Please note that since we aligned the time step of the returned path with the SAPIEN time step, we don’t need to interpolate the returned path.

In [ ]:
def follow_path(result, viewer): # parameter viewer is for later visualization
    n_step = result['position'].shape[0]
    for i in range(n_step):  
        qf = robot.compute_passive_force(
            gravity=True, 
            coriolis_and_centrifugal=True)
        robot.set_qf(qf)
        for j in range(7):
            active_joints[j].set_drive_target(result['position'][i][j])
            active_joints[j].set_drive_velocity_target(result['velocity'][i][j])
        scene.step()
        if i % 4 == 0:
            scene.update_render()
            viewer.render()

We also compensate the passive forces through `set_qf()` (see *Robotics Tutorial 1: Getting Started with Robot* for details).

You can also use your own controller.

> Note: If you find your robot doesn’t move as expected, please double-check your controller, especially the controller’s parameters. In many cases, the planner finds a good path while the controller fails to follow the path.

## Plan with screw motion

Besides using the sampling-based algorithms, we also provide another simple way (trick) to plan a path. For some tasks, we can directly move the `move_group` link towards the target pose. It’s internally achieved by first calculating the relative transformation from its current pose to the target pose, then calculating the relative transformation’s exponential coordinates, and finally calculating the joint velocities with the Jacobian matrix.

Compared to the sampling-based algorithms, planning with screw motion has the following pros:

faster: since it doesn’t need to sample lots of states in the joint space, planning with screw motion can save lots of planning time.

straighter path: there is no guarantee for sampling-based algorithms to generate straight paths even it’s a simple lifting task since it connects states in the joint space. In contrast, the returned path by the exponential coordinates and the Jacobian matrix can sometimes be more reasonable. See the above figures for comparison.

You can call `planner.plan_screw()` to plan a path with screw motion. Similar to `planner.plan()`, it also takes two required arguments: target pose and current joint positions, and returns a dict containing the same set of elements.

In [ ]:
def move_to_pose_with_screw(pose, viewer): # parameter viewer is for later visualization
    result = planner.plan_screw(pose, robot.get_qpos(), time_step=1/250)
    if result['status'] != "Success":
        result = planner.plan(pose, robot.get_qpos(), time_step=1/250)
        if result['status'] != "Success":
            print(result['status'])
            return -1 
    follow_path(result, viewer)
    return 0

However, planning with screw motion only succeeds when there is no collision during the planning since it can not detour or replan. We thus recommend use `planner.plan_screw()` for some simple tasks or combined with `planner.plan()`. As shown in the code, we first try `planner.plan_screw()`, if it fails (e.g., collision during the planning), we then turn to the sampling-based algorithms. Other arguments are the same with `planner.plan()`.

`planner.plan_screw()` also takes `qpos_step = 0.1`, `time_step = 0.1`, `use_point_cloud = False`, `use_attach = False`, and `verbose = False` as optional arguments, where `qpos_step` specifies the incremental distance of the joint positions during the path generation (before time paramtertization).

## Move the boxes

To control the gripper, we use set_drive_target() to set target positions for the two gripper joints:

In [ ]:
def open_gripper(viewer): # parameter viewer is for later visualization
    for joint in active_joints[-2:]:
        joint.set_drive_target(0.4)
    for i in range(100): 
        qf = robot.compute_passive_force(
            gravity=True, 
            coriolis_and_centrifugal=True)
        robot.set_qf(qf)
        scene.step()
        if i % 4 == 0:
            scene.update_render()
            viewer.render()

def close_gripper(viewer): # parameter viewer is for later visualization
    for joint in active_joints[-2:]:
        joint.set_drive_target(0)
    for i in range(100):  
        qf = robot.compute_passive_force(
            gravity=True, 
            coriolis_and_centrifugal=True)
        robot.set_qf(qf)
        scene.step()
        if i % 4 == 0:
            scene.update_render()
            viewer.render()

In this example, we manually mark some landmark poses to move the boxes:

In [ ]:
def move_to_pose(pose, with_screw, viewer): # parameter viewer is for later visualization
    if with_screw:
        return move_to_pose_with_screw(pose, viewer)
    else:
        return move_to_pose_with_RRTConnect(pose, viewer)

def demo(with_screw, viewer):
    poses = [[0.4, 0.3, 0.12, 0, 1, 0, 0],
            [0.2, -0.3, 0.08, 0, 1, 0, 0],
            [0.6, 0.1, 0.14, 0, 1, 0, 0]]
    for i in range(3):
        pose = poses[i]
        pose[2] += 0.2
        move_to_pose(pose, with_screw, viewer)
        open_gripper(viewer)
        pose[2] -= 0.12
        move_to_pose(pose, with_screw, viewer)
        close_gripper(viewer)
        pose[2] += 0.12
        move_to_pose(pose, with_screw, viewer)
        pose[0] += 0.1
        move_to_pose(pose, with_screw, viewer)
        pose[2] -= 0.12
        move_to_pose(pose, with_screw, viewer)
        open_gripper(viewer)
        pose[2] += 0.12
        move_to_pose(pose, with_screw, viewer)

## Visualization

Now we are ready to visualize the simulation! You can try to run the simulation loop with/without screw motion planning by specifying `with_screw` flag.

In [ ]:
viewer = Viewer(renderer)
viewer.set_scene(scene)
viewer.set_camera_xyz(x=1.2, y=0.25, z=0.4)
viewer.set_camera_rpy(r=0, p=-0.4, y=2.7)

demo(with_screw=True, viewer=viewer)

viewer.close()

## Additional Reading: Inverse Kinematics

Inverse kinematics determine the joint positions that provide the desired pose for the robot’s end-effectors. In mplib, you can solve the inverse kinematics of the `move_group` link with:

    planner.IK(target_pose, init_qpos, n_init_qpos = 20, threshold = 1e-3)

`Planner.IK()` internally implements a numerical method and takes the following arguments:

- `target_pose`: a 7-dim list specifies the target pose of the `move_group` link. The first three elements describe the position part, and the remaining four elements describe the quaternion (wxyz) for the rotation part.
- `init_qpos`: a list describes the joint positions of all the active joints (e.g., given by SAPIEN). It will be used as the initial state for the numerical method.
- `n_init_qpos=20`: besides the provided initial state, the method also samples extra initial states to run the algorithm for at most `n_init_qpos` times. In this way, it can avoid local minimums and increase the success rate.
- `threshold=1e-3`: a threshold for determining whether the calculated pose is close enough to to the target pose.

It returns a tuple of two elements:
- `status`: a string indicates the status.
- `result`: a NumPy array describes the calculated joint positions.

If `planner.IK()` fails, please increase `n_init_qpos` or double-check whether the target pose is reachable.